In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

class WebScraper:
    def __init__(self):
        # Initialize the Chrome WebDriver
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.headless = True
        self.driver = webdriver.Chrome(options=chrome_options)

        # URL of the website you want to scrape
        self.base_url = 'https://etenders.gov.in/eprocure/app'
        self.data_list = []

    def scrape_tender_data(self):
        try:
            # Navigate to the initial page
            self.driver.get(self.base_url)
            time.sleep(2)

            # Find the table with id "activeTenders"
            table_element = self.driver.find_element(By.ID, "activeTenders")

            # Find all the rows (tr elements) inside the table
            rows = table_element.find_elements(By.TAG_NAME, "tr")[0:]

            for row in rows:
                link_element = row.find_element(By.TAG_NAME, "a")
                link = link_element.get_attribute("href")
                link_text = link_element.text  # Get the link text before clicking

                self.driver.execute_script("window.open('', '_blank');")
                self.driver.switch_to.window(self.driver.window_handles[-1])
                self.driver.get(link)
                time.sleep(2)

                entry_data = self.extract_data_from_page()
                entry_data['url'] = link

                self.data_list.append(entry_data)

                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])
                time.sleep(2)

        except Exception as e:
            print(f"Error: {e}")
            self.driver.quit()

    def extract_data_from_page(self):
        data = {}
        try:
            data['Tender ID'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Tender ID")]/following-sibling::td/b').text
            data['Tender Title'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Title")]/following-sibling::td/b').text
            data['Reference No.'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Tender Reference Number")]/following-sibling::td/b').text
            data['Organisation Chain'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Organisation Chain")]/following-sibling::td/b').text
            data['Location'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Location")]/following-sibling::td').text
            data['E-Published Date'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Published Date")]/following-sibling::td').text
            data['Bid Submission Closing Date'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Submission End Date")]/following-sibling::td').text
            data['Tender Opening Date'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Bid Opening Date")]/following-sibling::td').text
            data['Tender Value in Rupees'] = self.driver.find_element(By.XPATH, '//td[contains(b, "Tender Value in ₹")]/following-sibling::td/b').text
        except Exception as e:
            print(f"Error extracting data: {e}")

        return data

    def save_to_csv(self):
        # Close the WebDriver when done
        self.driver.quit()

        # Convert the list of dictionaries into a Pandas DataFrame
        df = pd.DataFrame(self.data_list)

        # Save the DataFrame to a CSV file
        df.to_csv('tender_data.csv', index=False)

if __name__ == "__main__":
    scraper = WebScraper()
    scraper.scrape_tender_data()
    scraper.save_to_csv()


In [4]:
# Display the DataFrame
pd.read_csv('tender_data.csv')

,Tender ID,Tender Title,Reference No.,Organisation Chain,Location,E-Published Date,Bid Submission Closing Date,Tender Opening Date,Tender Value in Rupees,url
0,2023_YIL_170251_1,Renovation and Upgradation of Pond near Admin ...,9905/05/ND2305/MR/EOCIVIL/2023,Yantra India Limited||ORDNANCE FACTORY AMBAJHA...,"Ordnance Factory Ambajhari, Nagpur",10-Oct-2023 12:30 PM,31-Oct-2023 02:00 PM,01-Nov-2023 03:00 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
1,2023_MCD_170265_1,MCD/EE-(Pr.)-II/Sh.S/23-24/8-3,MCD/EE-(Pr.)-II/Sh.S/23-24/8-3,Municipal Corporation of Delhi||Engineering - ...,Mayur Vihar Phase-I,10-Oct-2023 12:00 PM,16-Oct-2023 02:00 PM,17-Oct-2023 02:05 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
2,2023_MCD_170264_1,MCD/EE-(Pr.)-II/Sh.S/23-24/8-2,MCD/EE-(Pr.)-II/Sh.S/23-24/8-2,Municipal Corporation of Delhi||Engineering - ...,Mayur Vihar Phase-I,10-Oct-2023 12:00 PM,16-Oct-2023 02:00 PM,17-Oct-2023 02:05 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
3,2023_EDMC_170275_1,CMC OF RO SYSTEM FOR A PERIOD OF ONE YEAR IN S...,sdnh/2023-24/154,East Delhi Municipal Corporation (EDMC)||SDN H...,SERVICES,10-Oct-2023 12:00 PM,20-Oct-2023 12:00 PM,21-Oct-2023 12:30 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
4,2023_EDMC_170269_1,CMC OF SUCTION MACHINE FOR A PERIOD OF ONE YEA...,sdnh/2023-24/153,East Delhi Municipal Corporation (EDMC)||SDN H...,works,10-Oct-2023 12:00 PM,20-Oct-2023 12:00 PM,21-Oct-2023 12:30 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
5,2023_MCD_170263_1,MCD/EE-(Pr.)-II/Sh.S/23-24/8-1,MCD/EE-(Pr.)-II/Sh.S/23-24/8-1,Municipal Corporation of Delhi||Engineering - ...,Patparganj,10-Oct-2023 12:00 PM,16-Oct-2023 02:00 PM,17-Oct-2023 02:05 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
6,2023_MUNPL_170258_1,Replacement of generator busduct CTs (Phase Si...,NIT/23-24/80 Dated 10.10.2023,Meja Urja Nigam Pvt Ltd||Meja Thermal Power Pr...,MUNPL Meja,10-Oct-2023 12:00 PM,20-Oct-2023 06:00 PM,21-Oct-2023 12:00 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
7,2023_BSNL_170262_1,Providing and fixing Lightening arrestors to c...,14/EE/BSNL/CD/LDH/2023-24 dt 09.10.2023,Bharat Sanchar Nigam Limited (Govt of India En...,Different locations in BSNL Bathinda OA,10-Oct-2023 11:30 AM,17-Oct-2023 06:00 PM,20-Oct-2023 11:30 AM,Product Category,https://etenders.gov.in/eprocure/app?component...
8,2023_GAIL_169836_1,RJ/B378-000-WL-MR-3379/279,RJ/B378-000-WL-MR-3379/279,GAIL (India) Limited||Project Management Consu...,GAIL-USAR,10-Oct-2023 11:00 AM,31-Oct-2023 02:00 PM,01-Nov-2023 02:00 PM,Product Category,https://etenders.gov.in/eprocure/app?component...
9,2023_MCD_170248_1,SHM pole with Light,MCD/TR/6849/2023_6_1_1/1,Municipal Corporation of Delhi||Engineering - ...,"Narela, RANI KHERA",10-Oct-2023 12:00 PM,17-Oct-2023 11:00 AM,17-Oct-2023 11:30 AM,Product Category,https://etenders.gov.in/eprocure/app?component...
